In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import random

In [2]:
document = ["Lawang-Sewu memiliki sistem pendingin air di bawah lantai, dan di atap. Itulah fungsi utama basement #lawangsewu",
            "lawang-sewu berada di ujung Bodjongweg atau yang sekarang dikenal dengan nama Jalan Pemuda",
            "lawang-sewu pernah digunakan sebagai penjara bawah tanah oleh serdadu Jepang.",
            "lawang-sewu merupakan landmark Kota Semarang yang paling dikenal para turis baik lokal maupun asing",
            "Lawang-Sewu adalah salah satu bangunan bersejarah yang dibangun oleh pemerintahan kolonial Belanda, pada 27 Februari 1904.",
            "lawang-sewu dulu menjadi lokasi pertempuran yang hebat antara pemuda Angkatan Muda Kereta Api melawan Kempetai dan Kidobutai Jepang",
            "Masyarakat setempat menyebutnya Lawang-Sewu (Seribu Pintu) dikarenakan bangunan tersebut memiliki pintu yang sangat banyak.",
            "Lawang-Sewu dibangun pada tahun 1904 dan selesai pada tahun 1907. Terletak di bundaran Tugu Muda yang dahulu disebut Wilhelminaplein.",
            "Lawang-Sewu tempo doeloe. Ada kereta api lewat juga di depannya.",
            "di lantai dua terdapat aula besar yg dulu digunakan sebagai ruang pesta. Di sudut aula terpasang wastafel yg sudah ada sejak pertma kali lawang-sewu dibangun."]
len(document)

10

# Preprocessing

In [3]:
# import
from preprocess.normalize import Normalize
from preprocess.tokenize import Tokenize
from preprocess.symspell import SymSpell

In [4]:
# initialize
normalizer = Normalize()
tokenizer = Tokenize()
symspell = SymSpell(max_dictionary_edit_distance=3)
symspell.load_complete_model_from_json('preprocess\data\corpus_complete_model.json', encoding="ISO-8859-1")

Loading dictionary...
Processing dictionary...
Copied 94811 words to master dictionary...
Copied 679534 hashes to master dictionary...


In [5]:
# do process
doc_preprocessed = []

for tweet in document:
    # normalize
    tweet_norm = normalizer.remove_ascii_unicode(tweet)
    tweet_norm = normalizer.remove_rt_fav(tweet_norm)
    tweet_norm = normalizer.lower_text(tweet_norm)
    tweet_norm = normalizer.remove_newline(tweet_norm)
    tweet_norm = normalizer.remove_url(tweet_norm)
    tweet_norm = normalizer.remove_emoticon(tweet_norm)
    tweet_norm = normalizer.remove_hashtag_mention(tweet_norm)
    tweet_norm = normalizer.remove_punctuation(tweet_norm)
    
    # tokenize
    tweet_tok = tokenizer.WordTokenize(tweet_norm, removepunct=True)
    
    # spell correction
    temp = []
    for token in tweet_tok:
        suggestion = symspell.lookup(phrase=token, verbosity=1, max_edit_distance=3)

        # option if there is no suggestion
        if len(suggestion) > 0:
            get_suggestion = str(suggestion[0]).split(':')[0]
            temp.append(get_suggestion)
        else:
            temp.append(token)
    tweet_prepared = ' '.join(temp)
    
    doc_preprocessed.append(tweet_prepared)

In [6]:
doc_preprocessed

['lawang-sewu memiliki sistem pendingin air di bawah lantai dan di atap itulah fungsi utama basement',
 'lawang-sewu berada di ujung bodjongweg atau yang sekarang dikenal dengan nama jalan pemuda',
 'lawang-sewu pernah digunakan sebagai penjara bawah tanah oleh serdadu jepang',
 'lawang-sewu merupakan landmark kota semarang yang paling dikenal para turis baik lokal maupun asing',
 'lawang-sewu adalah salah satu bangunan bersejarah yang dibangun oleh pemerintahan kolonial belanda pada 27 februari 1904',
 'lawang-sewu dulu menjadi lokasi pertempuran yang hebat antara pemuda angkatan muda kereta api melawan kempetai dan kidobutai jepang',
 'masyarakat setempat menyebutnya lawang-sewu seribu pintu dikarenakan bangunan tersebut memiliki pintu yang sangat banyak',
 'lawang-sewu dibangun pada tahun 1904 dan selesai pada tahun 907 terletak di bundaran tugu muda yang dahulu disebut wilhelminaplein',
 'lawang-sewu tempo doeloe ada kereta api lewat juga di depannya',
 'di lantai dua terdapat aula

# POSLDA
## HMM Tagger

In [7]:
# import
from hmmtagger.tagger import MainTagger
from tokenization import *

In [8]:
# initialize
tagger = MainTagger("resource/Lexicon.trn", "resource/Ngram.trn", 0, 3, 3, 0, 0, False, 0.2, 0, 500.0, 1)

In [9]:
# do process
doc_tagged = []

for tweet in doc_preprocessed:
    if len(tweet) == 0: continue
    out = sentence_extraction(cleaning(tweet))

    join_token = []
    for o in out:
        strtag = " ".join(tokenisasi_kalimat(o)).strip()
        join_token.extend(tagger.taggingStr(strtag))
    doc_tagged.append(' '.join(join_token))

In [10]:
doc_tagged

['lawang-sewu/NN memiliki/VBT sistem/NN pendingin/NN air/NN di/IN bawah/NN lantai/NN dan/CC di/IN atap/IN itulah/DT fungsi/NN utama/JJ basement/NN',
 'lawang-sewu/NN berada/VBI di/IN ujung/VBT bodjongweg/NN atau/CC yang/SC sekarang/NN dikenal/VBT dengan/IN nama/NN jalan/NN pemuda/NN',
 'lawang-sewu/NN pernah/RB digunakan/VBT sebagai/IN penjara/NN bawah/NN tanah/NN oleh/IN serdadu/NN jepang/NN',
 'lawang-sewu/NN merupakan/VBT landmark/NN kota/NN semarang/NN yang/SC paling/RB dikenal/VBT para/DT turis/NN baik/JJ lokal/NN maupun/CC asing/JJ',
 'lawang-sewu/NN adalah/VBT salah/JJ satu/CDP bangunan/NN bersejarah/VBI yang/SC dibangun/VBT oleh/IN pemerintahan/NN kolonial/JJ belanda/NN pada/IN 27/CDP februari/NN 1904/CDP',
 'lawang-sewu/NN dulu/RB menjadi/VBT lokasi/NN pertempuran/NN yang/SC hebat/JJ antara/IN pemuda/NN angkatan/NN muda/JJ kereta/NN api/NN melawan/VBT kempetai/NN dan/CC kidobutai/NN jepang/NN',
 'masyarakat/NN setempat/NN menyebutnya/VBT lawang-sewu/NN seribu/NN pintu/NN dikar

## Kelas Tag

In [11]:
# define
Ccon = ['JJ', 'NN','NNP', 'NNG', 'VBI', 'VBT']
Cfnc = ['OP', 'CP', 'GM', ';', ':', '"', '.',
         ',', '-', '...', 'RB', 'IN', 'MD', 'CC',
         'SC', 'DT', 'UH', 'CDO', 'CDC', 'CDP', 'CDI',
         'PRP', 'WP', 'PRN', 'PRL', 'NEG', 'SYM', 'RP', 'FW']

In [12]:
# do process
doc_classified = []

for tweet in doc_tagged:
    tweet_split = tweet.split(' ')
    
    temp = {"Content": [], "Function": []}
    con = []
    fnc = []
    
    for token in tweet_split:
        word = token.split('/', 1)[0]
        tag = token.split('/', 1)[1]
        
        if tag in Ccon:
            con.append(token)
        elif tag in Cfnc:
            fnc.append(token)
            
    temp["Content"].append(' '.join(con))
    temp["Function"].append(' '.join(fnc))
    
    doc_classified.append(temp)

In [13]:
doc_classified

[{'Content': ['lawang-sewu/NN memiliki/VBT sistem/NN pendingin/NN air/NN bawah/NN lantai/NN fungsi/NN utama/JJ basement/NN'],
  'Function': ['di/IN dan/CC di/IN atap/IN itulah/DT']},
 {'Content': ['lawang-sewu/NN berada/VBI ujung/VBT bodjongweg/NN sekarang/NN dikenal/VBT nama/NN jalan/NN pemuda/NN'],
  'Function': ['di/IN atau/CC yang/SC dengan/IN']},
 {'Content': ['lawang-sewu/NN digunakan/VBT penjara/NN bawah/NN tanah/NN serdadu/NN jepang/NN'],
  'Function': ['pernah/RB sebagai/IN oleh/IN']},
 {'Content': ['lawang-sewu/NN merupakan/VBT landmark/NN kota/NN semarang/NN dikenal/VBT turis/NN baik/JJ lokal/NN asing/JJ'],
  'Function': ['yang/SC paling/RB para/DT maupun/CC']},
 {'Content': ['lawang-sewu/NN adalah/VBT salah/JJ bangunan/NN bersejarah/VBI dibangun/VBT pemerintahan/NN kolonial/JJ belanda/NN februari/NN'],
  'Function': ['satu/CDP yang/SC oleh/IN pada/IN 27/CDP 1904/CDP']},
 {'Content': ['lawang-sewu/NN menjadi/VBT lokasi/NN pertempuran/NN hebat/JJ pemuda/NN angkatan/NN muda/JJ

In [14]:
# split document content and function
doc_content = []
for tweet in doc_classified:
    doc_content.append(''.join(tweet['Content']))

print(doc_content)
print()

# split tag and word
doc_prepared = []
for tweet in doc_content:
    tweet_split = tweet.split(' ')
    
    temp = []
    for token in tweet_split:
        word = token.split('/', 1)[0]
        temp.append(word)
    
    doc_prepared.append(temp)

print(doc_prepared)

['lawang-sewu/NN memiliki/VBT sistem/NN pendingin/NN air/NN bawah/NN lantai/NN fungsi/NN utama/JJ basement/NN', 'lawang-sewu/NN berada/VBI ujung/VBT bodjongweg/NN sekarang/NN dikenal/VBT nama/NN jalan/NN pemuda/NN', 'lawang-sewu/NN digunakan/VBT penjara/NN bawah/NN tanah/NN serdadu/NN jepang/NN', 'lawang-sewu/NN merupakan/VBT landmark/NN kota/NN semarang/NN dikenal/VBT turis/NN baik/JJ lokal/NN asing/JJ', 'lawang-sewu/NN adalah/VBT salah/JJ bangunan/NN bersejarah/VBI dibangun/VBT pemerintahan/NN kolonial/JJ belanda/NN februari/NN', 'lawang-sewu/NN menjadi/VBT lokasi/NN pertempuran/NN hebat/JJ pemuda/NN angkatan/NN muda/JJ kereta/NN api/NN melawan/VBT kempetai/NN kidobutai/NN jepang/NN', 'masyarakat/NN setempat/NN menyebutnya/VBT lawang-sewu/NN seribu/NN pintu/NN dikarenakan/VBT bangunan/NN memiliki/VBT pintu/NN banyak/JJ', 'lawang-sewu/NN dibangun/VBT tahun/NN selesai/NN tahun/NN terletak/VBI bundaran/NN tugu/NN muda/JJ dahulu/JJ disebut/VBT wilhelminaplein/NN', 'lawang-sewu/NN tempo/N

# LDA

In [15]:
# import
from lda.ldamodel import LdaModel

In [16]:
%%time
# iniatialize
k = 2
alpha = 0.01
beta = 0.01
iterations = 1000

# do process
lda = LdaModel(doc_prepared, k, alpha, beta, iterations)

Wall time: 2.15 s


In [17]:
result = lda.get_topic_word_pwz(doc_content)

df_lda = pd.DataFrame(result, columns=['Topik', 'Kata', 'PWZ'])

In [18]:
df_lda.head()

,Topik,Kata,PWZ
0,0,lawang-sewu/NN,0.105810
1,0,dibangun/VBT,0.052993
2,0,bangunan/NN,0.035387
3,0,memiliki/VBT,0.035387
4,0,pintu/NN,0.035387


In [19]:
print(df_lda[df_lda['Topik']==0]['PWZ'].to_string(index=False))

0.105810
0.052993
0.035387
0.035387
0.035387
0.035387
0.035387
0.035387
0.035387
0.017782
0.017782
0.017782
0.017782
0.017782
0.017782
0.017782
0.017782
0.017782
0.017782
0.017782


In [20]:
lda.perplexity()

5.016035741399591

# Grammar

In [21]:
from collections import defaultdict

dict_ldapwz = defaultdict(list)

In [22]:
for idx, row in df_lda.iterrows():
#     dict_ldapwz[row['Topik']].append(row['Kata'])
    dict_ldapwz[row['Topik']].append([row['Kata'], row['PWZ']])

In [23]:
dict_ldapwz

defaultdict(list,
            {0: [['lawang-sewu/NN', 0.10580985915492958],
              ['dibangun/VBT', 0.05299295774647887],
              ['bangunan/NN', 0.03538732394366197],
              ['memiliki/VBT', 0.03538732394366197],
              ['pintu/NN', 0.03538732394366197],
              ['tahun/NN', 0.03538732394366197],
              ['lantai/NN', 0.03538732394366197],
              ['bawah/NN', 0.03538732394366197],
              ['digunakan/VBT', 0.03538732394366197],
              ['sistem/NN', 0.01778169014084507],
              ['basement/NN', 0.01778169014084507],
              ['penjara/NN', 0.01778169014084507],
              ['jepang/NN', 0.01778169014084507],
              ['salah/JJ', 0.01778169014084507],
              ['bersejarah/VBI', 0.01778169014084507],
              ['pemerintahan/NN', 0.01778169014084507],
              ['belanda/NN', 0.01778169014084507],
              ['februari/NN', 0.01778169014084507],
              ['muda/JJ', 0.01778169014084507],
 

In [24]:
import re
from collections import OrderedDict, defaultdict

In [25]:
NP = ['_NN', '_NNG', '_NNP', '_NP _PP']
VP = ['_VBT _NN', '_VBT _NN _CC _NN', '_VBT _NP', '_VBI', '_SC _JJ']

In [26]:
def get_list_tag(dict_words_by_tag):
    result = []
    for key in dict_words_by_tag:
        result.append(key)
    return result

In [27]:
def generate_base_grammar(list_tag):
    result = {}
    
    if '_VBT' not in list_tag:
        list_tag.append('_VBT')
    
    S = {"_S": ["_NP _VP"]}
    PP = {"_PP": ["_IN _NP"]}
    
    if '_JJ' in list_tag:
        NP_RULES = generate_NP(list_tag)
        NP = {"_NP": NP_RULES}
        if check_VP(list_tag):
            VP_RULES = ['_VP _PP', '_JJ'] + generate_VP(list_tag)
            VP = {"_VP": VP_RULES}
        else:
            VP = {"_VP": ['_JJ']}

        for r in [S, NP, VP, PP]:
            result.update(r)
        return result
    else:
        NP_RULES = remove_JJ(generate_NP(list_tag))
        NP = {"_NP": NP_RULES}
        if check_VP(list_tag):
            VP_RULES = ['_VP _PP'] + remove_JJ(generate_VP(list_tag))
            VP = {"_VP": VP_RULES}
        else:
            VP = {"_VP": ['_PP']}
            
        for r in [S, NP, VP, PP]:
            result.update(r)
        return result

def generate_NP(list_tag):
    result = []
    for tag in list_tag:
        for words in NP:
            if re.search(r'\b' + tag + r'\b', words):
                result.append(words)
    return list(OrderedDict.fromkeys(result))
    
def check_VP(list_tag):
    for tag in list_tag:
        if 'V' in tag:
            return True
    return False

def generate_VP(list_tag):
    result = []
    for tag in list_tag:
        for words in VP:
            if re.search(r'\b' + tag + r'\b', words):
                result.append(words)
    return list(OrderedDict.fromkeys(result))

def remove_JJ(list_tag):
    result = []
    for tag in list_tag:
        if '_JJ' in tag:
            continue
        else:
            result.append(tag)
    return result

In [28]:
def generate_words_grammar(dict_words_by_tag):
    result = {}
    IN = {"_IN": ['di', 'sebagai', 'oleh']}
    CC = {"_CC": ['dan']}
    SC = {"_SC": ['yang']}
    ADD_VBT = {"_VBT": ['memiliki', 'adalah', 'merupakan', 'terdapat', 'yaitu', 'sebagai', 'mempunyai']}
    
    WORDS = dict_words_by_tag
    if '_VBT' in WORDS:
        for word in ADD_VBT["_VBT"]:
            if word not in WORDS['_VBT']:
                WORDS['_VBT'].append(word)
    else:
        WORDS.update(ADD_VBT)
            
    for r in [IN, CC, SC, WORDS]:
        result.update(r)
    return result

In [29]:
def organize_words_by_tag(list_words):
    result = defaultdict(list)
    
    i = []
    for s in list_words:
        word, pwz = s[0], s[1]
        
        wrd = word.split('/')[0]
        tag = word.split('/')[1]
        result['_'+tag].append([wrd, pwz])
    return dict(result)

In [30]:
def split_word_pwz(dict_words_pwz_by_tag):
    dict_word_by_tag = defaultdict(list)
    dict_pwz_by_tag = defaultdict(list)
    
    for key, values in dict_words_pwz_by_tag.items():
        for data in values:
            word, pwz = data[0], data[1]
            dict_word_by_tag[key].append(word)
            dict_pwz_by_tag[key].append(pwz)
    return dict(dict_word_by_tag), dict(dict_pwz_by_tag)

In [31]:
def create_grammar(dict_words_by_topic):
    grammar = {}
    
    dict_words_pwz_by_tag = organize_words_by_tag(dict_words_by_topic)
    list_tag = get_list_tag(dict_words_pwz_by_tag)
    base_grammar = generate_base_grammar(list_tag)
    dict_word_by_tag, dict_pwz_by_tag = split_word_pwz(dict_words_pwz_by_tag)
    words_grammar = generate_words_grammar(dict_word_by_tag)
    
    for r in [base_grammar, words_grammar]:
        grammar.update(r)
    return grammar, dict_pwz_by_tag 

In [32]:
def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z

In [33]:
def is_terminal(token):
    return token[0] != "_"

In [34]:
sys_random = random.SystemRandom()

def expand(grammar, tokens, dict_pwz_by_tag):
    for i, token in enumerate(tokens):

        # skip over terminals
        if is_terminal(token): continue

        # if we get here, we found a non-terminal token
        # so we need to choose a replacement at random
        replacement = sys_random.choice(grammar[token])
        
        if replacement == '_NN':
            weight = [x/sum(dict_pwz_by_tag['_NN']) for x in dict_pwz_by_tag['_NN']]
            replacement = nr.choice(grammar['_NN'], p=weight)
            
        if is_terminal(replacement):
            tokens[i] = replacement
        else:
            tokens = tokens[:i] + replacement.split() + tokens[(i+1):]
       
        # now call expand on the new list of tokens
        return expand(grammar, tokens, dict_pwz_by_tag)

    # if we get here we had all terminals and are done
    return tokens

In [35]:
def generate_sentence(grammar, dict_pwz_by_tag):
    return expand(grammar, ["_S"], dict_pwz_by_tag)

In [36]:
def create_sentences_from_data(dict_data):
    result = {}
    for topic, words in dict_data.items():
        sentence = []
        grammar, dict_pwz_by_tag = create_grammar(words)
        print(grammar)
        for s in range(10):
            sentence.append(' '.join(generate_sentence(grammar, dict_pwz_by_tag)))
        result = merge_two_dicts(result, {topic: sentence})
    return result

In [68]:
dict_story = create_sentences_from_data(dict(dict_ldapwz))
# for _ in dict_story.items():
#     print(_)

{'_S': ['_NP _VP'], '_NP': ['_NN'], '_VP': ['_VP _PP', '_JJ', '_VBT _NN', '_VBT _NN _CC _NN', '_VBT _NP', '_SC _JJ', '_VBI'], '_PP': ['_IN _NP'], '_IN': ['di', 'sebagai', 'oleh'], '_CC': ['dan'], '_SC': ['yang'], '_NN': ['lawang-sewu', 'bangunan', 'pintu', 'tahun', 'lantai', 'bawah', 'sistem', 'basement', 'penjara', 'jepang', 'pemerintahan', 'belanda', 'februari', 'setempat'], '_VBT': ['dibangun', 'memiliki', 'digunakan', 'adalah', 'merupakan', 'terdapat', 'yaitu', 'sebagai', 'mempunyai'], '_JJ': ['salah', 'muda'], '_VBI': ['bersejarah']}
{'_S': ['_NP _VP'], '_NP': ['_NN'], '_VP': ['_VP _PP', '_JJ', '_VBT _NN', '_VBT _NN _CC _NN', '_VBI', '_VBT _NP', '_SC _JJ'], '_PP': ['_IN _NP'], '_IN': ['di', 'sebagai', 'oleh'], '_CC': ['dan'], '_SC': ['yang'], '_NN': ['lawang-sewu', 'kereta', 'api', 'aula', 'pemuda', 'sekarang', 'nama', 'landmark', 'turis', 'lokal', 'pertempuran', 'angkatan', 'kempetai'], '_VBI': ['ada', 'berada'], '_VBT': ['dikenal', 'merupakan', 'menjadi', 'memiliki', 'adalah', '

In [69]:
dict_story

{0: ['lawang-sewu mempunyai jepang',
  'bangunan bersejarah',
  'pemerintahan memiliki lawang-sewu dan pemerintahan sebagai lantai',
  'lawang-sewu yang salah',
  'belanda merupakan lantai dan bawah',
  'lantai mempunyai bawah',
  'lawang-sewu muda',
  'jepang yang muda',
  'belanda salah',
  'pintu bersejarah'],
 1: ['lawang-sewu berada',
  'angkatan ada di api',
  'pemuda mempunyai lawang-sewu',
  'lawang-sewu ada',
  'pemuda berada',
  'sekarang asing di lawang-sewu',
  'aula berada',
  'kereta ada',
  'lawang-sewu sebagai lokal',
  'aula adalah kereta']}

In [39]:
elements = ['one', 'two', 'three'] 
weights = [0.001, 0.009, 0.001]
new_weight = [x/sum(weights) for x in weights]

from numpy.random import choice
# print(choice(elements, p=new_weight))
print(new_weight)

[0.09090909090909091, 0.8181818181818181, 0.09090909090909091]
